In [2]:
import math

In [3]:
def B1(thickness, TVL): # Calculates B factor using only 1 TVL. Important for PATIENT SCATTER
    '''
    param thickness: thickness of barrier (cm)
    param TVL: TVL (cm) where it is ALWAYS dependent on energy. Can also depend on the angle too
    '''
    B = 10**-(thickness/TVL)
    return B

def B2(thickness, TVL1, TVLe): # Calculates B factor using 2 TVLs. Important for LEAKAGE
    '''
    param thickness: thickness of barrier (cm)
    param TVL1: first TVL (cm)
    param TVLe: equilibrium TVL (cm)
    '''
    B = 10**-(1+((thickness-TVL1)/TVLe))
    return B

def H_ps_unshielded(W, d_secondary, d_scatter, a, F): # Calculates unshielded equivalent dose for PATIENT SCATTER
    '''
    param W: workload in a typical week (Gy/wk)
    param d_secondary: distance from isocenter to the point of interest (cm). Includes the 0.3 m
    param d_scatter: distance from target to isocenter. Usually 1 m
    param a: scatter fraction dependent on angle (deg) and beam energy (MV)
    param F: field size in cm^2
    '''
    H_ps_uns = (W * a * F) / (d_secondary**2 * d_scatter**2 * 400) * 10**3
    return H_ps_uns # (mSv/wk)

def H_leak_unshielded(W_L, d_leakage): # Calculates UNSHIELDED equivalent dose for LEAKAGE
    '''
    param W_L: leakage workload (Gy/wk)
    param d_leakage: distance from source of leakage to point of protection (cm)
    '''
    H_leak_uns = ((W_L*10**-3) / d_leakage**2) * 10**3 # 10**-3 is the included barriers from the head and 10**3 converts Sv to mSv
    return H_leak_uns # (mSv/wk)

def convert_to_meters(foot,inches):
    m = (foot * 12 * 2.54 + inches * 2.54) * 10e-3
    return m

def verify_2barrier(P, T, thickness,d,tvl_6mv_ps,tvl_18mv_ps,a_6mv,a_18mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param tvl_18mv_ps: TVL for 18 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    param a_18mv: scatter fraction for 18 MV
    '''
  
    H_l_6mv = H_leak_unshielded(1990, d) * B2(thickness, 34, 29)
    H_l_18mv = H_leak_unshielded(250, d) * B2(thickness, 36, 34)

    H_ps_6mv = H_ps_unshielded(520, d, 1, a_6mv, 1600) * B1(thickness, tvl_6mv_ps)
    H_ps_18mv = H_ps_unshielded(250, d, 1, a_18mv, 1600) * B1(thickness, tvl_18mv_ps)

    H_total = H_ps_6mv + H_ps_18mv + H_l_6mv + H_l_18mv
    
    if H_total < P /T:
        print("PASS")
    else:
        print("FAIL")
    print("Total dose: ", H_total, "mSv/wk")
    print("Dose limit: ", P / T, "mSv/wk")
    print(f"Distance {d} m")
    print(f"thickness used: {thickness} cm")

def verify_2barrier_vault2(P, T, thickness,d,tvl_6mv_ps,a_6mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    '''
  
    H_l_6mv = H_leak_unshielded(1435.5, d) * B2(thickness, 34, 29)

    H_ps_6mv = H_ps_unshielded(319, d, 1, a_6mv, 400) * B1(thickness, tvl_6mv_ps)

    H_total = H_ps_6mv + H_l_6mv 
    if H_total < P /T:
        print("PASS")
    else:
        print("FAIL")
    print(f"H_ps_6mv: {H_ps_6mv} mSv/wk")
    print(f"H_l_6mv: {H_l_6mv} mSv/wk")
    print("------------------------------------")
    print("Total dose: ", H_total, "mSv/wk")
    print("Dose limit: ", P / T, "mSv/wk")
    print('-------------------------------------------')
    print(f"Thickness used: {thickness} cm = {thickness /2.54} inches")

# Important note: thickness must be in centimeters.

In [4]:
# Vault #1 Wall A Secondary Barrier
# Worst case scenario of 33.37 deg barely grazing the edge of primary barrier
# 8.57258 m distance
# 2'3"
verify_2barrier(0.02,(1/40),68.58,8.57258702,26,32,2.77e-3,2.53e-3)

PASS
Total dose:  0.6394931153452919 mSv/wk
Dose limit:  0.7999999999999999 mSv/wk
Distance 8.57258702 m
thickness used: 68.58 cm


In [5]:
# Vault #1 Wall C Secondary Barrier
# Worst case scenario of 33.92 deg barely grazing the edge of primary barrier
# 8.627792 m distance
# 2'11" = 88.9 cm

verify_2barrier(0.02,(1/40),convert_to_meters(2,11)*100,8.627792498,26,32,2.77e-3,2.53e-3)

PASS
Total dose:  0.12966064256015258 mSv/wk
Dose limit:  0.7999999999999999 mSv/wk
Distance 8.627792498 m
thickness used: 88.9 cm


In [6]:
# Vault #1 Wall B Secondary Barrier
# Worst case scenario of 90 deg incidence
# 3'7" = 109.22 cm
verify_2barrier(0.02,1,convert_to_meters(3,7)*100,convert_to_meters(20,11.81056),17,19,4.26e-4,1.89e-4)

PASS
Total dose:  0.016702701784852704 mSv/wk
Dose limit:  0.02 mSv/wk
Distance 6.395988224 m
thickness used: 109.22 cm


# Vault # 1: Projected distance for calculating the ceiling primary barrier width

In [7]:
x=convert_to_meters(17,9.04728)
y=convert_to_meters(15,5)

hyp = (x**2 + y**2)**.5
print(f"Projected distance for ceiling 1 barrier width = {hyp + 1} m")
ceiling_width = 0.4 * math.sqrt(2) * (hyp + 1) + 0.6
print(f"Ceiling width = {ceiling_width} m = {ceiling_width*100/2.54} inches")

Projected distance for ceiling 1 barrier width = 8.16685850497932 m
Ceiling width = 5.219872823889526 m = 205.50680409013881 inches


# Vault 2: Projected distance for calculating the ceiling primary barrier width 

height = 13'-11' and
iso to proximal surface of wall C = 22'-6"

In [8]:
x2 = convert_to_meters(22,6)
y2 = convert_to_meters(13,11)

hyp2 = (x2**2+y2**2)**0.5
print(f"Projected distance for ceiling 1 barrier width = {hyp2 + 1} m")
ceiling_width_vault2 = 0.2 * math.sqrt(2) * (hyp2 + 1) + 0.6
print(f"Ceiling width = {ceiling_width_vault2} m = {ceiling_width_vault2*100/2.54} inches")


Projected distance for ceiling 1 barrier width = 9.063809970479216 m
Ceiling width = 3.1636325974048387 m = 124.55246446475743 inches


# Vault 1: Verification for Ceiling Secondary Radiation

In [9]:
# Worst case 34.79 degree scatter
# It passes if we use oblique thickness
testvalue = convert_to_meters(3,1.74691)*100 # cm
print(testvalue)
# Fails if we use the perpendicular thickness, 78.74 cm
verify_2barrier(0.02,0.025,testvalue,convert_to_meters(16,2.59264),26,32,2.77e-3,2.53e-3)

95.8771514
PASS
Total dose:  0.23051605159644686 mSv/wk
Dose limit:  0.7999999999999999 mSv/wk
Distance 4.942653056 m
thickness used: 95.8771514 cm


# Vault 2: Verification for Ceiling Secondary Radiation

Oblique thickness was used. A guess-and-check method was used (iterative method). A line was drawn to intersect through the primarry barrier at 24 degrees.

In [15]:
# Worst case 24.00 degree scatter
d_ceil_vault2 = convert_to_meters(13,10.59086)

verify_2barrier_vault2(0.02,0.025,d_ceil_vault2, convert_to_meters(2,1.58964)*100,34,6.73e-3)

PASS
H_ps_6mv: 0.3815546843639258 mSv/wk
H_l_6mv: 0.36116910170480987 mSv/wk
------------------------------------
Total dose:  0.7427237860687357 mSv/wk
Dose limit:  0.7999999999999999 mSv/wk
-------------------------------------------
Thickness used: 4.2314078440000005 cm = 1.6659086 inches


# Vault 2: Verification with only Wall Z such that it barely grazes IMW - Storage Room

We will use the storage room as a controlled area since it is adjacent to the treatment room and exposure must be monitored closely. Also notice that the storage area can only be accessed through the LINAC controle console, which is guaranteed to be a controlled area...

In [10]:
# Worst case 57.18 scatter angle where it does not touch IMW
testvalue2 = convert_to_meters(36,10.44416) # m
print(f"{testvalue2} m")
verify_2barrier_vault2(0.1, 1/20, convert_to_meters(1,7)*100,testvalue2,23,1.39e-3)
print('----------------------------')
print("Storage areas use T = 1/20")

11.238081664 m
PASS
H_ps_6mv: 0.028000142811191325 mSv/wk
H_l_6mv: 0.3663487310404552 mSv/wk
------------------------------------
Total dose:  0.3943488738516465 mSv/wk
Dose limit:  2.0 mSv/wk
-------------------------------------------
Thickness used: 48.260000000000005 cm = 19.0 inches
----------------------------
Storage areas use T = 1/20


# Vault 2: Verification where it passes through corner of Wall Z, giving the smallest scatter angle - Storage Room

We will use the storage room as a controlled area since it is adjacent to the treatment room and exposure must be monitored closely. Also notice that the storage area can only be accessed through the LINAC controle console, which is guaranteed to be a controlled area...

In [11]:
# 54.30 deg scatter 
d_wallz_vault2_corner = convert_to_meters(38,1.82978)
verify_2barrier_vault2(0.1,1/20,convert_to_meters(1,7)*100,d_wallz_vault2_corner,23,1.39e-3)

PASS
H_ps_6mv: 0.02614984408250735 mSv/wk
H_l_6mv: 0.3421397619694756 mSv/wk
------------------------------------
Total dose:  0.36828960605198297 mSv/wk
Dose limit:  2.0 mSv/wk
-------------------------------------------
Thickness used: 48.260000000000005 cm = 19.0 inches


# Vault 1: Verification with only Wall Z such that it barely grazes IMW - QA Room

P = controlled -> Only workers that are specifically trained in using ionizing radiation are allowed. A medical physics resident was consulted at MBPCC and informed that it is mainly a storage area where they come and go in minutes. A simple storage area.

Because it is adjacent to the treatment room, exposure should be monitored closely, and thus, access is limited for radiation protection purposes.

T = 1/5. 5 hrs/wk is very conservative. 

Oblique thickness is used.

In [12]:
# Worst case 57.29 deg scatter angle

d_wallz_vault1 = convert_to_meters(36,9.87206)
print(f"d_wallz_vault1 = {d_wallz_vault1} m")
verify_2barrier(0.1,1/8,convert_to_meters(1,10.58097)*100,d_wallz_vault1,23,27,1.39e-3,8.64e-4)

d_wallz_vault1 = 11.223550324 m
PASS
Total dose:  0.4191835529118715 mSv/wk
Dose limit:  0.8 mSv/wk
Distance 11.223550324 m
thickness used: 57.3556638 cm


# Vault 1: Verification with Wall Z through the corner - QA Room

Oblique thickness is used with worst case 54.42 deg scatter angle. Same logic for P and T as above

In [13]:
# Worst case 54.52 deg scatter angle
# P = controlled
# T = 1/8 

d_wallz_vault1_corner = convert_to_meters(30,11.81287)
verify_2barrier(0.1,1/8,convert_to_meters(1,11.33242)*100,d_wallz_vault1_corner,23,27,1.39e-3,8.64e-4)

PASS
Total dose:  0.5058977504942629 mSv/wk
Dose limit:  0.8 mSv/wk
Distance 9.444046898 m
thickness used: 59.2643468 cm
